In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import os
import imageio
warnings.filterwarnings("ignore")

from segysak.segy import (
    segy_loader,
    get_segy_texthead,
    segy_header_scan,
    segy_header_scrape,
    well_known_byte_locs,
)

import ipywidgets as widgets
from ipywidgets import interactive

path_data = './data/'

def load_seismic_data(path):
    # Load seismic data
    cube = segy_loader(path+'volve10r12-full-twt-sub3d.sgy', **well_known_byte_locs('petrel_3d'))

    # Calculate corner points of seismic survey
    cube.seis.calc_corner_points()
    corners = np.array(cube.attrs["corner_points_xy"])

    return cube, corners

def load_horizon_data(path):
    # Load horizon
    hrz = pd.read_csv(path+'/hor_twt_hugin_fm_top.dat', names=['cdp_x', 'cdp_y', 'twt'], sep='\s+')
    return hrz

def map_horizon_to_seismic(cube, hrz):
    # Map horizon to seismic survey
    hrz_mapped = cube.seis.surface_from_points(hrz, "twt", right=("cdp_x", "cdp_y"))
    return hrz_mapped

def plot_seismic_section(cube, corners, hrz_mapped, inline_val):
    opt = dict(
        x="xline",
        y="twt",
        add_colorbar=True,
        interpolation="spline16",
        robust=True,
        yincrease=False,
        cmap="RdBu",
        vmin=-5,
        vmax=5,
        alpha=0.8
    )
    f, ax = plt.subplots(1,2,figsize=(20,5))
    # ... [The rest of the plotting code from the f(inline_val) function]
    tform = cube.seis.get_affine_transform()
    axs = ax[0]
    mesh = axs.pcolormesh(
        hrz_mapped.iline,
        hrz_mapped.xline,
        hrz_mapped.twt.T,
        shading="auto",
        transform=tform + axs.transData,
        cmap = 'viridis_r'
    )
    axs.set_aspect(1)
    axs.axis('off')
    dx = 5 # number of lines in x and y grid
    line_ext = 5 #extension of grid lines
    text_ext = 5

    axs.plot(corners[:,0], corners[:,1], lw = 4, c='white')
    axs.plot([corners[0,0]  ,corners[-1,0]], [corners[0,1]  ,corners[-1,1]],  lw = 4, c='white')
    #Grid 
    seis_grid_x = np.round(np.linspace(cube.iline.min(),cube.iline.max(), dx).astype('int64'),-1)
    seis_grid_y = np.round(np.linspace(cube.xline.min(),cube.xline.max(), dx).astype('int64'),-1)

    props = dict( facecolor='white',edgecolor = 'k', alpha=1)

    _ = axs.plot([inline_val, inline_val], [cube.xline[0], cube.xline[-1]], transform=tform + axs.transData, color="k", lw = 2)
    axs.text(inline_val-5, cube.xline[-1]+6, transform=tform + axs.transData, s=str(inline_val), rotation= 75, fontsize = 8, bbox = props)

    #Grid loop:
    for i in range(seis_grid_x.shape[0]):
        # Inline grid
        _ = axs.plot([cube.iline[0]-line_ext, cube.iline[-1]+line_ext], [seis_grid_y[i], seis_grid_y[i]], transform=tform + axs.transData, color="grey", lw = .5)
        #Crossline grid
        _ = axs.plot([seis_grid_x[i], seis_grid_x[i]], [cube.xline[0]-line_ext, cube.xline[-1]+line_ext], transform=tform + axs.transData, color="grey", lw = .5)
        #Inline numbers
        #axs.text(seis_grid_x[i], cube.xline[-1]+10, transform=tform + axs.transData, s=str(seis_grid_x[i]), rotation= 75, fontsize = 8)
        #Crossline numbers
        axs.text(cube.iline[-1]+text_ext, seis_grid_y[i]+5, transform=tform + axs.transData, s=str(seis_grid_y[i]), rotation= 75 - 90, fontsize = 10)

    #'Inline #' label
    axs.text(seis_grid_x[len(seis_grid_x)//2]-5, cube.xline[-1]+20, transform=tform + axs.transData, s='Inline #', rotation= 75  , fontsize = 10);
    #'Crossline #' label
    axs.text(cube.iline[-1]+10, seis_grid_y[len(seis_grid_y)//2]+8, transform=tform + axs.transData, s='Crossline #', rotation= 75 - 90  , fontsize = 10);
    

    ####### vertical seismic plot here ######

    cube.data.sel(iline=inline_val, twt=slice(2300, 3000)).plot.imshow(ax=ax[1],   **opt )
    x, t = hrz_mapped.sel(iline=inline_val).xline, hrz_mapped.sel(iline=inline_val).twt
    ax[1].plot(x, t, color="k", lw = 3)
    ax[1].invert_xaxis()
    ax[1].set_xlabel('Crossline #')
    ax[1].set_ylabel('TWT (ms)')
    ax[1].set_title('Inline #'+str(inline_val))
    ax[1].text(x[11],t[0], s = 'Top Hugin', bbox = props, fontsize = 6)
def create_interactive_plot(cube, corners, hrz_mapped):
    interactive_plot = interactive(lambda inline_val: plot_seismic_section(cube, corners, hrz_mapped, inline_val),
                                   inline_val=(cube.iline.data.min(), cube.iline.data.max()))
    output = interactive_plot.children[-1]
    output.layout.height = '480px'
    return interactive_plot

if __name__ == "__main__":
    cube, corners = load_seismic_data(path_data)
    hrz = load_horizon_data(path_data)
    hrz_mapped = map_horizon_to_seismic(cube, hrz)
    interactive_plot = create_interactive_plot(cube, corners, hrz_mapped)
    display(interactive_plot)


  0%|          | 0.00/12.3k [00:00<?, ? traces/s]

Loading as 3D
Fast direction is TRACE_SEQUENCE_FILE


Converting SEGY:   0%|          | 0.00/12.3k [00:00<?, ? traces/s]

interactive(children=(IntSlider(value=10120, description='inline_val', max=10150, min=10090), Output(layout=La…

In [6]:
import pyvista as pv
import numpy as np

def create_seismic_volume_mesh(cube):
    data = cube.data.values
    data_normalized = (data - np.min(data)) / (np.max(data) - np.min(data))

    # Create a 3D uniform grid from seismic data
    grid = pv.UniformGrid()
    grid.dimensions = np.array(data_normalized.shape) + 1
    grid.spacing = [1, 1, 1]
    grid.origin = [0, 0, 0]
    grid.cell_data["values"] = data_normalized.flatten(order="F")

    return grid


def create_horizon_mesh(hrz_mapped):
    X, Y, Z = np.meshgrid(hrz_mapped.iline, hrz_mapped.xline, hrz_mapped.twt, indexing='ij')
    points = np.column_stack((X.ravel(), Y.ravel(), Z.ravel()))

    # Create a mesh from horizon data
    mesh = pv.PolyData(points)
    mesh = mesh.delaunay_2d()

    return mesh

def plot_seismic_volume_horizon(cube, hrz_mapped):
    seismic_volume_mesh = create_seismic_volume_mesh(cube)
    horizon_mesh = create_horizon_mesh(hrz_mapped)

    plotter = pv.Plotter(notebook=True)  # Set notebook=False for an interactive window
    plotter.add_mesh(seismic_volume_mesh, opacity="linear", cmap="viridis")
    plotter.add_mesh(horizon_mesh, color="green", show_edges=True)

    plotter.show_grid()
    plotter.show()

if __name__ == "__main__":
    cube, corners = load_seismic_data(path_data)
    hrz = load_horizon_data(path_data)
    hrz_mapped = map_horizon_to_seismic(cube, hrz)

    plot_seismic_volume_horizon(cube, hrz_mapped)


  0%|          | 0.00/12.3k [00:00<?, ? traces/s]

Loading as 3D
Fast direction is TRACE_SEQUENCE_FILE


Converting SEGY:   0%|          | 0.00/12.3k [00:00<?, ? traces/s]

: 

: 